In [ ]:
from sklearn.externals import joblib
from keras.models import load_model, model_from_json
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
#define a model name to load
model_name = 'model_50K'

In [ ]:
eng_tokenizer = joblib.load('data/' + model_name + '/eng_tokenizer.pkl')
eng_length = joblib.load('data/' + model_name + '/eng_length.pkl')
eng_vocab_size = len(eng_tokenizer.word_counts) + 1
targ_tokenizer = joblib.load('data/' + model_name + '/targ_tokenizer.pkl')
targ_length = joblib.load('data/' + model_name + '/targ_length.pkl')
targ_vocab_size = len(targ_tokenizer.word_counts) + 1
# trainX = joblib.load('data/' + model_name + '/trainX.pkl')
# testX = joblib.load('data/' + model_name + '/testX.pkl')
# model = load_model('data/' + model_name + '/' + model_name + '.h5')

In [ ]:
model.save_weights('data/' + model_name + '/weights.h5')

In [ ]:
model_json = model.to_json()
with open('data/' + model_name + '/model.json', "w") as json_file:
    json_file.write(model_json)

In [ ]:
json_file = open('data/' + model_name + '/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
loaded_model.load_weights('data/' + model_name + '/weights.h5')

In [ ]:
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
loaded_model.summary()

In [1]:
def prepare_model(model_name):
    
    from sklearn.externals import joblib
    from keras.models import model_from_json
    
    json_file = open('data/' + model_name + '/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights('data/' + model_name + '/weights.h5')
    loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return loaded_model

In [2]:
model = prepare_model('model_50K')

/Users/sonik/anaconda3/envs/pythondata/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 256)           3161344   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 20, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20, 512)           1050624   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 21581)         11071053  
Total params: 15,808,333
Trainable params: 15,808,333
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def encode_input(input_text, tokenizer=eng_tokenizer, leng=eng_length):
    input_text = [input_text]
    token = tokenizer.texts_to_sequences(input_text)
    token = pad_sequences(token, maxlen=leng, padding='post')
    return token

def translate(model, token):
    for i, tok in enumerate(token):
        tok = tok.reshape((1, tok.shape[0]))
        translation = predict_sequence(model, targ_tokenizer, tok)
        print(translation)
        
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [np.argmax(vector) for vector in prediction]
    target = []
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

In [ ]:
translate(model, encode_input('who likes it'))